In [154]:
import random
import networkx as nx
import itertools

In [155]:
def load_graph_from_file(file_path):
    G = nx.Graph() 

    with open(file_path, "r") as file:
        for line in file:
            parts = line.split()
            if parts[0] == 'e':
                node1 = int(parts[1]) 
                node2 = int(parts[2])  
                G.add_edge(node1, node2) 
    return G

In [156]:
def is_resolving_set(B, G):
    distances = {}
    for v in G.nodes:
        distances[v] = [nx.shortest_path_length(G, v, u) for u in B]
    
    for v1, v2 in itertools.combinations(G.nodes, 2):
        if distances[v1] == distances[v2]:
            return False
    return True

In [157]:
def fitness(resolving_set,G,shortest_paths):
    unresolved_pairs = 0
    for u, v in itertools.combinations(G.nodes, 2):
        if not any(abs(shortest_paths[u][j] - shortest_paths[v][j]) > 0 for j in resolving_set if j in shortest_paths[u]):
            unresolved_pairs += 1  # Par (u, v) nije razrešen

    return len(resolving_set) + 100 * unresolved_pairs  # Penalizacija nerešenih parova

In [158]:
def construct_solution(G,pheromones,heuristics,alpha,beta,shortest_paths):
    resolving_set = set()
    available_nodes = set(G.nodes)

    while available_nodes:
        # Računanje verovatnoće izbora svakog čvora
        probabilities = {node: (pheromones[node] ** alpha) * (heuristics[node] ** beta) for node in available_nodes}
        total = sum(probabilities.values())
        
        if total == 0:
            break
        
        # Normalizacija verovatnoća
        probabilities = {node: prob / total for node, prob in probabilities.items()}

        # Izbor čvora proporcionalno verovatnoći
        selected_node = random.choices(list(probabilities.keys()), weights=list(probabilities.values()))[0]
        resolving_set.add(selected_node)
        available_nodes.remove(selected_node)

        # Ako je rešenje dovoljno dobro, prekidamo ranije
        if fitness(resolving_set,G,shortest_paths) < 100:
            break

    return resolving_set

In [159]:
def update_pheromones(best_solution,pheromones,evaporation,Q,G,shortest_paths):
    # Isparavanje feromona
    for node in pheromones:
        pheromones[node] *= (1 - evaporation)
    
    # Dodavanje novih feromona na čvorove iz najboljeg rešenja
    for node in best_solution:
        pheromones[node] += Q / fitness(best_solution,G,shortest_paths)

In [160]:
def ant_colony_optimization(G,num_iterations,num_ants,alpha,beta,evaporation,pheromones,heuristics,Q,shortest_paths):
    best_solution = None
    best_fitness = float("inf")

    for iteration in range(num_iterations):
        solutions = [construct_solution(G,pheromones,heuristics,alpha,beta,shortest_paths) for _ in range(num_ants)]
        sorted_solutions = sorted(solutions, key=lambda s: fitness(s, G,shortest_paths))        
        if fitness(sorted_solutions[0],G,shortest_paths) < best_fitness:
            best_solution = sorted_solutions[0]
            best_fitness = fitness(best_solution,G,shortest_paths)

        update_pheromones(best_solution,pheromones,evaporation,Q,G,shortest_paths)
        
        print(f"Iteracija {iteration}, najbolji skup {best_fitness},  {best_solution}")
    print("\nOptimalni rešavajući skup:", best_solution)
    return best_solution

In [161]:
# G = nx.Graph()
# G.add_edges_from([(1,2),(1,3),(1,5),(1,6),(2,4),(2,6),(3,5),(3,4),(4,5),(4,6),(5,6)])

# G = nx.petersen_graph()

G = load_graph_from_file('grafovi//gcol//gcol1.txt')

n = len(G.nodes)

# Parametri ACO algoritma
num_ants = 50     # Broj mrava po iteraciji
num_iterations = 50  # Broj iteracija
alpha = 0.5       # Uticaj feromona
beta = 1.0        # Uticaj heuristike (udaljenosti)
evaporation = 0.5  # Isparavanje feromona
Q = 100           # Konstanta za ažuriranje feromona

# Inicijalizacija feromona (svaki čvor dobija početnu vrednost)
pheromones = {node: 1.0 for node in G.nodes}

# Heuristička vrednost: obrnuta prosečna udaljenost od svih drugih čvorova
heuristics = {node: 1.0 / (sum(nx.shortest_path_length(G, node, j) for j in G.nodes if j != node) + 1) for node in G.nodes}


shortest_paths = dict(nx.all_pairs_shortest_path_length(G))  # Svi putevi između svih parova


rez = ant_colony_optimization(G,num_iterations,num_ants,alpha,beta,evaporation,pheromones,heuristics,Q,shortest_paths)


Iteracija 0, najbolji skup 10,  {32, 1, 35, 7, 8, 73, 13, 78, 26, 92}
Iteracija 1, najbolji skup 10,  {32, 1, 35, 7, 8, 73, 13, 78, 26, 92}
Iteracija 2, najbolji skup 10,  {32, 1, 35, 7, 8, 73, 13, 78, 26, 92}
Iteracija 3, najbolji skup 10,  {32, 1, 35, 7, 8, 73, 13, 78, 26, 92}
Iteracija 4, najbolji skup 10,  {32, 1, 35, 7, 8, 73, 13, 78, 26, 92}


KeyboardInterrupt: 

In [ ]:
is_resolving_set(rez,G)

True